<a href="https://colab.research.google.com/github/emilycalvert/scripts_cite/blob/main/verficar_tallando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alistar entorno.

In [ ]:
!pip install numpy
!pip install geopy
!pip install pandas
!pip install pyproj
!pip install geemap
!pip install timedelta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for timedelta: filename=timedelta-2020.12.3-py3-none-any.whl size=1554 sha256=a63532f1edacd1b905dd7df67b33122436dbdba6f82fd37525686fadbdf98a86
  Stored in directory: /root/.cache/pip/wheels/be/c0/1f/843bd73679dc7dfb26758c4effc6e273d3eebf2da2a35082b1
Successfully built timedelta


In [ ]:
import ee
import time
import geemap
import datetime
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pyproj import Transformer, Proj, transform
from dateutil.relativedelta import relativedelta

# **1. Autorizar e inicializar Earth Engine**
#### _Autenticar e inicializar la API de Google Earth Engine (GEE)._




- Conecte e inicialice con tu cuenta de earth engine y proyecto.

In [ ]:
ee.Authenticate(auth_mode="notebook")
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wz9oKilqz3_LHE8gVNcMZBGGPnI5QdvHKO3kLBx891M&tc=TswcRvmz9HVfHWl6wGvAupSVNrgn_w5429x0xA0hLeY&cc=31bqnR3mBTTu0Ihdgi9Uc9Q-7gKlMNqEzAkO5JNG6LA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AeaYSHDzkbvhqqO5WLc2BPO8jg7qkdN0g4UVGc6poaZh__QmdrGOmJeWEyk

Successfully saved authorization token.


# **2. Cargar CSV**

#### _Cargue el archivo CSV que contiene las coordenadas y analícelo para crear puntos._

- Exporta tu hoja de cálculo con los vértices como csv. La integridad y uniformidad del documento original es importante. Si tiene dificultades en este paso, consulte el curso [aquí.](https://www.youtube.com/watch?v=uGx0PHD6o9M)

- Al usar este cuaderno en colab. Habrá un panel a la izquierda. Si no estás en los archivos. Haga clic en el icono de la carpeta donde los botones están al ras de la pantalla. Ahora donde dice "Archivos", haz clic en el ícono de archivo con una flecha en el medio y sube tus vértices. Alternativamente, puede arrastrar y soltar en el panel de archivos.

In [204]:
# @title
csv_viaje = 'imazacoord_NOppi.csv'  # Una vez que haya cargado su archivo, en esta línea cambie el nombre por el nombre del archivo que cargó con la extensión.
df = pd.read_csv(csv_viaje)

In [205]:
# @title
df.head()

,NCONTRA,NOMTIT,NUMRUC,RESAPR,ESTE,NORTE,VERTICE
0,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,643560.25,9530613.00,1
1,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,643560.25,9527597.60,2
2,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,641579.82,9528512.95,3
3,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,640464.65,9528511.51,4
4,16-IQU/C-J-105-04,INDUSTRIAL MADERERA ZAPOTE S.A.,20103979529,RSD N° 019-2012-GRL-GGR-PRMRFFS-DER-SDPM,638560.25,9526722.71,5


In [206]:
# Aquí necesitamos proyectar desde UTM 18S a WGS84 (el CRS utilizado por Earth Engine).
transformer = Transformer.from_crs("epsg:32718", "epsg:4326", always_xy=True)
'''
Tenga en cuenta que si sus coordenadas no están en UTM 18S,
deberá consultar la documentación de pyproj y ajustar "epsg:32718"
al epsg que utilizan sus coordenadas. Aquí está el sitio donde
puedes encontrar esa información:
https://pyproj4.github.io/pyproj/stable/api/crs/crs.html
https://epsg.io/
'''
def utm_to_latlon(easting, northing):
    # Dado que transformador.transform(este, norte) ya se alinea con (x, y) => (ESTE, NORTE),
    # y desea regresar (latitud, longitud), asegúrese de capturar la salida correctamente.
    lon, lat = transformer.transform(easting, northing)
    return lat, lon  # Esto garantiza que la devolución esté en formato (latitud, longitud).

# Assuming 'df' is your DataFrame
df['latitud'], df['longitud'] = zip(*df.apply(lambda row: utm_to_latlon(row['ESTE'], row['NORTE']), axis=1)) # Donde dice 'ESTE' y 'NORTE', esto coincide de manera muy idéntica con los nombres de las columnas de latitud y longitud en sus datos.

#### Procesando

In [207]:
coordenadas = df[['latitud', 'longitud']].values.tolist()

In [208]:
puntos = [ee.Feature(ee.Geometry.Point([lon, lat])) for lat, lon in coordenadas]
puntos_fc = ee.FeatureCollection(puntos)

In [209]:
polygon = ee.Geometry.Polygon([[[lon, lat] for lat, lon in coordenadas]])

In [210]:
centroid = polygon.centroid(maxError=1)

#### Modificar

In [211]:
print(centroid.getInfo())

{'type': 'Point', 'coordinates': [-73.72766753816298, -4.349379692587443]}


In [212]:
centro= (-73.72766753816298, -4.349379692587443) # Aquí pegue las coordenadas impresas en la celda de arriba para definir el centro de su parcela.

# **3. Obtener imágenes:**

#### _Utilice la API de Earth Engine para consultar imágenes para las fechas y la cobertura de nubes especificadas._

In [213]:
antes = '2020-01-06'
'''
Establezca esto en una fecha anterior a la emisión del permiso.
Encontrará la primera imagen de satélite en esa fecha o antes que
cumpla con sus requisitos de cobertura de nubes.
'''
despues = '2023-01-06'
'''
Esta fecha es posterior a la cosecha y encontrará
una imagen para la fecha más cercana pero no anterior
a la cosecha que cumpla con el umbral de nubosidad.
'''
limite_coberatura_nubes = 20  # Establezca esto en su umbral máximo de cobertura de nubes.

In [214]:
antes_fecha = datetime.strptime(antes, '%Y-%m-%d')
antes1= antes_fecha + relativedelta(years=-1)

In [215]:
ahora= datetime.now()
ahora = ahora.strftime('%Y-%m-%d')

In [216]:
# @title
# Imagen antes de la emisión del permiso.
image1 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
           .filterDate(antes1, antes) \
           .filterBounds(puntos_fc) \
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', limite_coberatura_nubes)) \
           .sort('CLOUDY_PIXEL_PERCENTAGE') \
           .first()\
           .clip(polygon)

In [217]:
nir1 = image1.select('B8')
red1 = image1.select('B4')

ndvi1 = nir1.subtract(red1).divide(nir1.add(red1)).rename('NDVI')

In [218]:
date = ee.Date(image1.get('system:time_start')).format('YYYY-MM-dd')
print('Date of the image1 image:', date.getInfo())

Date of the image1 image: 2019-08-09


In [219]:
# @title
ndvi_antes = ndvi1.reduceRegion(
    reducer=ee.Reducer.mean(),
    scale=30,
    maxPixels=1e9
)

ndvi_antes = ndvi_antes.get('NDVI').getInfo()

print('NDVI Antes:', ndvi_antes)

NDVI Antes: 0.8255680912098503


In [220]:
# @title
# Aquí está la foto después de la cosecha.
image2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
           .filterDate(despues, ahora) \
           .filterBounds(puntos_fc) \
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', limite_coberatura_nubes)) \
           .sort('CLOUDY_PIXEL_PERCENTAGE') \
           .first()\
           .clip(polygon)

In [221]:
nir2 = image2.select('B8')
red2 = image2.select('B4')

ndvi2 = nir2.subtract(red2).divide(nir2.add(red2)).rename('NDVI')

In [222]:
# Print the date of the image2 image
date_image2 = ee.Date(image2.get('system:time_start')).format('YYYY-MM-dd')
print('Date of the image2 image:', date_image2.getInfo())

Date of the image2 image: 2023-09-17


In [223]:
# @title
ndvi_despues = ndvi2.reduceRegion(
    reducer=ee.Reducer.mean(),
    scale=30,
    maxPixels=1e9
)

ndvi_despues = ndvi_despues.get('NDVI').getInfo()

print('NDVI Despues:', ndvi_despues)

NDVI Despues: 0.8451962613228494


#### Imagenes

In [224]:
image1

In [225]:
  ndvi1

In [226]:
image2

In [227]:
ndvi2

#### Estilo

In [231]:
vizParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2500,
    'gamma': 1.0
}

## **Antes**

In [232]:
# @title
# Aquí vemos la trama previa a la emisión del permiso.
# https://colab.research.google.com/github/giswqs/gee-tutorials/blob/master/Image/image_visualization.ipynb#scrollTo=dIqOCYLbo0SF


Map = geemap.Map()


Map.centerObject(polygon, 11)
Map.addLayer(image1, vizParams, 'Antes Permiso')

Map

Map(center=[-4.349379692587443, -73.72766753816298], controls=(WidgetControl(options=['position', 'transparent…

# **4. Calcular NDVI:**
#### _Calcular y enmascarar áreas donde se excede el umbral._

In [233]:
ndvi_diff = ndvi2.subtract(ndvi1).rename('Cambio')

loss_threshold = -0.1
'''
Es importante monitorear para asegurarse de que el umbral dado
refleje con precisión el área cosechada.
Normalmente, un área deforestada estaría en el rango de |0,1 a 0,2|
para el umbral del NDVI. Es posible que deba ajustar el script y
probarlo un poco desde aquí para encontrar las áreas deforestadas
para su parcela, ya que el área y las tendencias generales del NDVI
pueden afectar esto.

NOTA IMPORTANTE: Si el NDVI antes es menor que el NDVI después indica un aumento en la vegetación. Establezca el umbral en un número negativo. Se aplica lo inverso.
'''
loss_mask = ndvi_diff.lt(loss_threshold)

In [234]:
pixelArea = ee.Image.pixelArea().mask(loss_mask)

totalLossArea = pixelArea.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=polygon,
    scale=30,
    maxPixels=1e9
).get('area').getInfo()

print(f"Areas de Perdido de Vegetacion: {totalLossArea} metricos cubiertos")

Areas de Perdido de Vegetacion: 971196.2902937346 metricos cubiertos


In [235]:
# @title
vectorizedAndSimplified = loss_mask.updateMask(loss_mask).reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e9
).map(lambda feature: ee.Feature(feature.geometry().simplify(maxError=ee.ErrorMargin(1))))

indices = ee.List.sequence(0, vectorizedAndSimplified.size().subtract(1))

def assign_names_and_calculate_areas(feature, index):
    name = ee.String('Polygonio_').cat(ee.Number(index).add(1).format())
    area = feature.geometry().area(maxError=ee.ErrorMargin(1))
    return feature.set({'name': name, 'area_m2': area})

featuresWithInfo = ee.FeatureCollection(indices.map(lambda index: assign_names_and_calculate_areas(ee.Feature(vectorizedAndSimplified.toList(vectorizedAndSimplified.size()).get(index)), index)))

def print_feature_info(feature):
    feature = ee.Feature(feature)
    name = feature.get('name').getInfo()
    area = feature.get('area_m2').getInfo()
    vertices = feature.geometry().coordinates().getInfo()
    print(f"Nombre: {name}, Area (metricos cubiertos): {area}, Vertices: {vertices}")

featureList = featuresWithInfo.toList(featuresWithInfo.size())

# **5. Visualizar**
#### _Muestra los polígonos identificados sobre la imagen más reciente._

In [238]:
# @title
for i in range(featureList.size().getInfo()):
    print_feature_info(featureList.get(i))

Nombre: Polygonio_1.0, Area (metricos cubiertos): 904.2966507757552, Vertices: [[[-73.79609113958851, -4.300692817559388], [-73.79609071358944, -4.300964161326636], [-73.79582041496711, -4.300963733629746], [-73.79582084105601, -4.3006923898804645], [-73.79609113958851, -4.300692817559388]]]
Nombre: Polygonio_2.0, Area (metricos cubiertos): 904.2935115112335, Vertices: [[[-73.79593602907842, -4.398376447052391], [-73.79593559331467, -4.398647790100988], [-73.79566525988261, -4.398647352648394], [-73.79566569574519, -4.398376009626747], [-73.79593602907842, -4.398376447052391]]]
Nombre: Polygonio_3.0, Area (metricos cubiertos): 904.2980830559322, Vertices: [[[-73.79511736558696, -4.231769752215438], [-73.79511694610966, -4.232041096377945], [-73.79484667167895, -4.232040675175875], [-73.79484709125508, -4.231769331040317], [-73.79511736558696, -4.231769752215438]]]
Nombre: Polygonio_4.0, Area (metricos cubiertos): 904.2987304196871, Vertices: [[[-73.79488807778823, -4.205177596574901], 

In [237]:
# @title
styledPolygons = vectorizedAndSimplified.style(
    color='FFCC00',
    width=.75,
    fillColor='00000000',
    lineType='solid'
)

Map = geemap.Map()

Map.centerObject(polygon, 11)

Map.addLayer(image2, vizParams, 'Despues Tallando')

Map.addLayer(styledPolygons, {}, 'Mask Outline')

Map

Map(center=[-4.349379692587443, -73.72766753816298], controls=(WidgetControl(options=['position', 'transparent…